# Cencosud Desafío Data Science
**Autor**: Javier Martínez 

In [1]:
import pandas as pd
import numpy as np

from ml_utils.utils import *
import os
import warnings

## Lectura de los datos

In [25]:
# Lectura de la data
pd_data = pd.read_csv('./data/datos_ventas_test1.csv')
pd_data.head()

,ds,hierarchy_2_code,item_id,y
0,2021-10-10,82,1088128,1.000
1,2021-06-12,20,1147321,1.000
2,2021-09-20,10,190535,10.008
3,2021-07-10,82,949665,1.000
4,2021-02-09,83,1085445,1.000


Supuestos:
- ds = fecha
- y = venta
- producto_id = item_id
- hierarchy_2_code = rubro_id

In [26]:
pd_data = pd_data.rename(columns={  "ds": "fecha",
                                    "y": "venta",
                                    "producto_id":"item_id",
                                    "hierarchy_2_code":"rubro_id"})\
                    .sort_values(['fecha','item_id'])\
                    .copy()

pd_data['fecha'] = pd.to_datetime(pd_data['fecha'])

pd_data.head()

,fecha,rubro_id,item_id,venta
85320,2021-01-02,72,162,3.0
86394,2021-01-02,61,701,7.0
428980,2021-01-02,61,702,5.0
429332,2021-01-02,60,738,1.0
401258,2021-01-02,54,763,3.0


Cantidad de items

In [27]:
len(pd_data.item_id.unique())

6029

Periodo de tiempo

In [28]:
print("Mínima fecha",pd_data.fecha.min())
print("Máxima fecha",pd_data.fecha.max())

Mínima fecha 2021-01-02 00:00:00
Máxima fecha 2022-01-09 00:00:00


¿Qué rubro se lleva la mayor venta?

In [36]:
pd_venta_rubro = pd_data.groupby('rubro_id',as_index=False).agg({'venta':'sum'})
max_venta = pd_venta_rubro.venta.max()
pd_venta_rubro[pd_venta_rubro.venta==max_venta]

,rubro_id,venta
3,15,1367690.061


Calculando el número de observaciones para cada item

In [29]:
# Rango de fecha deseado
date_range = pd.date_range(start=pd_data.fecha.min(),
                           end=pd_data.fecha.max(),
                           #periods=None,
                           freq='D')

len(date_range)

373

Supuesto: se utilizarán items donde exista como mínimo un 90% de la información para la serie temporal.

In [23]:
# Agrupamiento por item
pd_n_item = pd_data.groupby('item_id',as_index=False).count()

# Selección de item_id validos
porc_valido = 0.9
items_validos = pd_n_item.query(f"fecha>{porc_valido*len(date_range)}")\
                            .item_id\
                            .unique()

items_no_validos = pd_n_item.query(f"fecha<={porc_valido*len(date_range)}")\
                            .item_id\
                            .unique()

# Data con item_id no validos
pd_no_model = pd_data[pd_data.item_id.isin(items_no_validos)]
pd_no_model.to_csv('./data/items_no_validos.csv',index=False)

# Data con item_id validos
pd_model = pd_data[pd_data.item_id.isin(items_validos)]
pd_model.to_csv('./data/items_validos.csv',index=False)

pd_model.head()

,fecha,rubro_id,item_id,venta
86394,2021-01-02,61,701,7.0
238854,2021-01-02,54,771,15.0
217054,2021-01-02,54,777,15.0
351303,2021-01-02,71,1278,115.0
302925,2021-01-02,71,1281,3.0


In [8]:
print('Número de Items con poca imformación temporal:',len(items_no_validos))
print('Número de Items suficiente imformación temporal:',len(items_validos))

Número de Items con poca imformación temporal: 5735
Número de Items suficiente imformación temporal: 294


## Reconstrucción de series temporales

Para cada uno de los items con más del 90% de la información (items_validos) se crea una serie temporal con fecha de inicio '2021-01-02' y fecha de finalización '2022-01-09'. Los datos faltantes son calculados mediante una interpolación lineal, este método es seleccionado dado los alcances de esta evaluación en la práctica es un criterio que implica mayor evaluación.

In [22]:
# items a reconstruir
item_id_list = list(pd_model.item_id.unique())


#-------------
#Funcion para el proceso de reconstruccion de las series temporales
def create_time_serie(item_id=701,
                      date_range=date_range,
                      pd_model=pd_model):
    """
    Función para crear series temporales de los items selecionados
    """

    data = pd.merge(pd.DataFrame(date_range,columns=['fecha']),
                    pd_model.query(f"item_id=={item_id}").copy(),
                    on=['fecha'],
                    how='left'
                    ).sort_values('fecha')

    # interpolacion para Nas
    data['venta'] = data['venta'].interpolate()
    data['rubro_id'] = data[data.rubro_id.notnull()].rubro_id.unique()[0]
    data['item_id'] = item_id
    
    return data
#-------------

# Aplicando proceso
pd_rebuild = pd.concat(
                    list(map(lambda item_id: create_time_serie(item_id,date_range=date_range, pd_model=pd_model),item_id_list ))
                    )

# Guardando datos
pd_rebuild.to_csv('./data/rebuild.csv',index=False)                    
pd_rebuild.head()

,fecha,rubro_id,item_id,venta
0,2021-01-02,61.0,701,7.0
1,2021-01-03,61.0,701,8.0
2,2021-01-04,61.0,701,11.0
3,2021-01-05,61.0,701,9.0
4,2021-01-06,61.0,701,7.0


## Entrenamiento de modelos

Ajuste de una Red neuronal Recurrente (LSTM) a cada una de las series con items válidos. Inicialmente se aplica una transformación Logarítmica para estabilizar la varianza y posteriormente se aplica la transformación minimax con la finalidad de llevar los valores a una escala cero uno. Este tipo de redes por lo general tienen un buen performance al utilizar este tipo de escalas. Las clases programadas estan disponibles en **ml_utils/utils** tal que:

1. **LogMinimax**: Transformación Logarítmica y minimax.
2. **RNN_LSTM**: Modelo de redes neuronales recurrentes (LSTM) programado en Keras (tensorflow).

In [10]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

In [12]:
list_items_validos = list(pd_rebuild.sort_values('item_id').item_id.unique())[29:]

#----------
def training_model(item_id,pd_rebuild=pd_rebuild,patience=20, epochs=100):
    """
    Función para el entrenamiento de modelos RNN LSTM
    """
    try: 
        test = RNN_LSTM(item_id=item_id, pd_rebuild=pd_rebuild)
        test.create_data()
        test.fit_model(patience=patience, epochs=epochs)
        test.validation_data()
        test.experimento_pd.to_csv(f'./models/experimento_pd_{test.item_id}.csv')
        test.pd_summary.to_csv(f'./models/pd_summary_{test.item_id}.csv')
    except:
        pass
#----------

# Iniciando proceso de entrenamiento a cada item
models = list(map(lambda item_id: training_model(item_id,pd_rebuild=pd_rebuild,patience=5, epochs=100), list_items_validos ))

## Guardando información

Guardando Métricas de entrenamiento

In [21]:
import os

DIR = './models/'

metricas = [DIR + x for x in os.listdir(DIR) if x.find('experimento_pd')!=-1]
predicciones = [DIR + x for x in os.listdir(DIR) if x.find('pd_summary')!=-1]

# metricas
pd_metricas = pd.concat([pd.read_csv(x) for x in metricas])
pd_metricas.to_csv('./data/metricas.csv',index=False)

# predicciones
pd_forecast = pd.concat([pd.read_csv(x) for x in predicciones])
pd_forecast.to_csv('./data/forecast.csv',index=False)
